# 関数定義

各ノートブック（*.ipynb）から利用する関数定義。  
[import-ipynb](https://pypi.org/project/import-ipynb/) を利用してipynbファイルをimportできるようにしている。


In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

%matplotlib inline

## 設定

In [2]:
def setup_seed(seed):
    """
    各種乱数系のseed値を設定する
    --------------------------------
    seed: シード値
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)  # Python
    np.random.seed(seed)  # Numpy
    tf.random.set_seed(seed)  # TensorFlow

## 音声ファイル操作

In [3]:
def normalize_fn(clip):
    rng = max(np.abs(np.min(clip)), np.max(clip))  # プラスマイナス1の範囲に広げる
    return clip / rng


def load_audio_clips(
    filename: str,
    interval: float = 2.0,
    duration: float = 0.5,
    normalize: bool = True,
    squeeze: bool = True,
) -> [np.ndarray]:
    """
    音声ファイルを読み込み、複数クリップに分割して返す
    --------------------------------
    filename: ファイル名
    interval: 音を分割するインターバル時間（秒）
    duration: 抽出する音の長さ（秒）
    normalize: -1..1の範囲にノーマライズするかどうか
    squeeze: サイズ1の次元を削除するかどうか
    --------------------------------
    return: 分割した複数の音声クリップ
    """

    # 音声ファイルの読み込み
    audio = tfio.audio.AudioIOTensor(filename)
    rate = audio.rate.numpy()
    data = audio.to_tensor().numpy().astype(np.float32)  # shape=(samples,1)
    if squeeze:
        data = data.squeeze()

    # 指定間隔ごとにデータを分割（指定サイズに満たない余りは切り捨てる）
    interval_step = int(rate * interval)
    duration_step = int(rate * duration)
    limit = data.size - duration_step + 1
    clips = []
    for split_pos in np.arange(0, limit, interval_step, dtype=np.int):
        clips.append(data[split_pos : split_pos + duration_step].astype(np.float32))

    # 各クリップ単位にノーマライズ (-1..1)
    if normalize:
        # rng = np.abs(audio.dtype.min)  # 24bitはint32になる
        clips = list(map(normalize_fn, clips))

    # 情報表示
    print(
        "{} {}kHz, clips:{}, duration:{}, normalize:{}".format(
            filename, rate / 1000, len(clips), duration_step, normalize
        )
    )
    return clips

In [4]:
def show_audio_clip(clips: [np.ndarray], pos: int, rate: int = 48000):
    """
    音声クリップの波形グラフと再生ボタンを表示する。
    --------------------------------
    clips: load_audio_clipsで読み込んだ音声クリップのリスト
    pos: 対象クリップの要素番号
    rate: サンプリングレート
    """

    # 波形グラフ
    data = clips[pos]
    plt.figure(figsize=(12, 3))
    librosa.display.waveplot(data)
    plt.show()

    # 再生ボタン
    ipd.display(ipd.Audio(data, rate=rate))

## ファイル操作

In [5]:
def readlines_as_list(filename: str, enc="utf_8_sig"):
    """
    ファイルを読み込んで、各行をリストで返す。
    --------------------------------
    filename: ファイル名
    enc: エンコード（default=UTF-8 BOM付き）
    --------------------------------
    return: 読み取った各行（list形式）
    """
    with open(filename, mode="r", encoding=enc) as f:
        return [s.strip() for s in f.readlines()]


def writelines(filename: str, lines: [str], enc="utf_8_sig"):
    """
    リスト文字列をファイルに出力する。
    --------------------------------
    filename: ファイル名
    lines: リスト文字列
    enc: エンコード（default=UTF-8 BOM付き）
    """
    with open(filename, mode="w", encoding=enc) as f:
        f.writelines("\n".join(lines))

## リスト処理など

In [6]:
def string_to_list(s: str, dtype: np.dtype = np.int64) -> np.ndarray:
    """
    文字列を1文字ずつ分解してdtype型のリストにキャストして返す。
    数字文字列を数値のリストに変換する用途で利用。
    --------------------------------
    s: 文字列
    dtype: 変換する型（default=np.int64）
    """
    return np.array(list(s)).astype(dtype)


def list_map(func, values):
    """
    values全てにfuncを適用して、結果をndarray型で返す。
    --------------------------------
    func: 適用する処理
    values: リスト
    """
    return np.array(list(map(func, values)))

## TensorFlow関連

In [7]:
def cardinality(ds: tf.data.Dataset):
    """
    tf.dada.Dataset.cardinality と同様の機能。
    オリジナル版は-2を返すことがあるため、更にラップしたもの。
    """
    cnt = ds.cardinality().numpy()
    if cnt == -2:
        cnt = len(list(ds.as_numpy_iterator()))
    return cnt

In [8]:
def show_history_graph(
    history,
    acc_key="accuracy",
    val_acc_key="val_accuracy",
    loss_key="loss",
    val_loss_key="val_loss",
):
    """
    Historyのグラフを描画。
    """
    fig = plt.figure(figsize=(13, 5), constrained_layout=True)
    ax = fig.subplots(1, 2)
    t_col = "#03F"
    v_col = "#F33"

    ax[0].set_title("Accuracy", fontsize=16)
    ax[0].plot(history.history[acc_key], color=t_col, lw=2, label=acc_key)
    ax[0].plot(history.history[val_acc_key], color=v_col, lw=2, label=val_acc_key)
    ax[1].set_title("Loss", fontsize=16)
    ax[1].plot(history.history[loss_key], color=t_col, lw=2, label=loss_key)
    ax[1].plot(history.history[val_loss_key], color=v_col, lw=2, label=val_loss_key)

    x_pos = np.arange(0, len(history.epoch), 1)
    for x in ax:
        x.set_xticks(x_pos)
        x.set_xticklabels(x_pos + 1)
        x.set_xlabel("Epoch")
        x.grid(True)
        x.legend(fontsize=12)

    plt.show()